# Setup

In [12]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import json
import numpy as np
import pandas as pd
import pvlib
import pytz
import folium
import geopy.distance
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()
NREL_API_KEY = os.getenv("NREL_API_KEY")
NREL_API_EMAIL = os.getenv("NREL_API_EMAIL")

In [3]:
import sys
sys.path.append('/Users/andrebach/code/solmap')
%load_ext autoreload
%autoreload 2

In [4]:
import data_generation

# Run it

In [265]:
congressional_summary_df = data_generation.geometry.load_congresional_district_points(
    file_to_load="congressional_geometry.csv",
)

In [268]:
generation_df = data_generation.solar.generate_values_at_points(
    congressional_summary_df
)

i = 0
i = 50
Cache miss at 58.63942421657351, -141.66555845649208: computing from scratch.
Missing solar weather for 58.63942421657351, -141.66555845649208
i = 100
i = 150
i = 200
i = 250
i = 300
i = 350
i = 400
Saved to generation_2024_11_16T12_51_25_733719.csv


In [107]:
data_generation.solar.save_as_geojson(congressional_summary_df, generation_df)

Missing values for 1 geos, continuing.
Saved to solar_points_2024_11_16T09_28_03_554544.json


True

In [227]:
# El Paso, west Texas
lat_source = 32.05526076086956
lon_source = -110.2840603695652

In [294]:
generation_df_el_paso = data_generation.solar.simulate_for_constant_clouds(
    lat_source,
    lon_source,
    congressional_summary_df,
    #limit_query="timestamp < '2019-02-01 12:00:00+00:00'",
)

i = 0
i = 50
Cache miss at 58.63942421657351, -141.66555845649208: computing from scratch.
Missing solar weather for 58.63942421657351, -141.66555845649208
i = 100
i = 150
i = 200
i = 250
i = 300
i = 350
i = 400
Saved to generation_clouds_matching_lat32.05526076086956_lon-110.2840603695652_2024_11_16T13_44_36_341365.csv


In [297]:
generation_df_el_paso.generation.min(), generation_df_el_paso.generation.max()

(234.19352779461238, 275.07116242249424)

In [298]:
generation_df.generation.min(), generation_df.generation.max()

(150.5035880476227, 274.7849533764213)

In [301]:
df = generation_df_el_paso
ncolors = 20
palette = sns.color_palette("flare_r", ncolors)
hex_colors = palette.as_hex()
vmin = 140
vmax = 285
vrange = vmax - vmin
df["color"] = df.generation.apply(
    lambda x: hex_colors[int((ncolors - 1) * (x - vmin) / vrange)]
)
m = folium.Map(
    location=(df.lat.mean(), df.lon.mean()),
    zoom_start=4,
    height=650,
    width=1083,
)
#folium.TileLayer('cartodbdark_matter').add_to(m)
for i, row in df.iterrows():
    folium.CircleMarker(
        location=[row.lat, row.lon],
        fill=True,
        stroke=False,
        fill_opacity=0.8,
        radius=7,
        color=row.color,
        tooltip=f"{row.GEO_ID}, {row.generation:0.1f}",
        popup=f"{row.GEO_ID}, {row.generation:0.1f}, {row.lat}, {row.lon}",
    ).add_to(m)
m

# End of notebook